In [28]:
import json
from pathlib import Path

import pandas as pd

In [29]:
from mhqa.dvc import load_experiments

filepaths = list(Path("../../tmp/frames-mini/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

5 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'validation'}},
  'retrieve': {'model_type': 't5', 'top_k': 3},
  'qa': {'model': 'llama-3.3-70b',
   'temperature': 0.1,
   'technique': 'multihop-decompose'},
  'run': 1},
 'metrics': {}}

In [30]:
for exp in experiments:
    train_params = exp["params"]["train"]
    if train_params.get("ensemble") is None:
        train_params["ensemble"] = "no"

experiments[0]

{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'validation'}},
  'retrieve': {'model_type': 't5', 'top_k': 3},
  'qa': {'model': 'llama-3.3-70b',
   'temperature': 0.1,
   'technique': 'multihop-decompose'},
  'run': 1},
 'metrics': {}}

In [31]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df


5 experiments before preprocessing


,id,name,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,params.retrieve.model_type,params.retrieve.top_k,params.qa.model,params.qa.temperature,params.qa.technique,params.run,metrics.exact_match,metrics.f1,metrics.0hops.exact_match,metrics.0hops.f1
0,workspace,None,bdsaglam/musique-sweep,answerable,train,noop,no,bdsaglam/musique-sweep,answerable,validation,t5,3,llama-3.3-70b,0.1,multihop-decompose,1,NaN,NaN,NaN,NaN
1,30027cd1a73a2d74acdd721adefe95e24cd3a0c1,pasty-sley,bdsaglam/frames-musique-mini,default,test[:20],miprov2-light,no,bdsaglam/frames-musique-mini,default,test[20:],golden,1,qwen-2.5-32b,0.0,cot,1,0.4125,0.549838,0.4125,0.549838
2,88cf8455a91f0046db02ed19cd4a7fb66218e705,surer-sike,bdsaglam/frames-musique-mini,default,test[:20],noop,no,bdsaglam/frames-musique-mini,default,test[20:],golden,1,qwen-2.5-32b,0.5,cot,1,0.4000,0.537584,0.4000,0.537584
3,d79cf4b97af41843834820c8c05f1e0f9f623f19,nifty-muss,bdsaglam/frames-musique-mini,default,test[:20],miprov2-light,no,bdsaglam/frames-musique-mini,default,test[20:],golden,1,qwen-2.5-32b,0.5,cot,1,0.3500,0.495338,0.3500,0.495338
4,046701ca67fe51827ed9d781ae4c45e3804e4f4f,least-lich,bdsaglam/frames-musique-mini,default,test[:20],noop,no,bdsaglam/frames-musique-mini,default,test[20:],golden,1,qwen-2.5-32b,0.0,cot,1,0.3875,0.535115,0.3875,0.535115


In [32]:
mask = df["params.evaluation.dataset.path"] == "bdsaglam/frames-musique-mini"

df = df.loc[mask].copy()

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")
df

4 experiments after preprocessing


,id,name,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,params.retrieve.model_type,params.retrieve.top_k,params.qa.model,params.qa.temperature,params.qa.technique,params.run,metrics.exact_match,metrics.f1,metrics.0hops.exact_match,metrics.0hops.f1
1,30027cd1a73a2d74acdd721adefe95e24cd3a0c1,pasty-sley,bdsaglam/frames-musique-mini,default,test[:20],miprov2-light,no,bdsaglam/frames-musique-mini,default,test[20:],golden,1,qwen-2.5-32b,0.0,cot,1,0.4125,0.549838,0.4125,0.549838
2,88cf8455a91f0046db02ed19cd4a7fb66218e705,surer-sike,bdsaglam/frames-musique-mini,default,test[:20],noop,no,bdsaglam/frames-musique-mini,default,test[20:],golden,1,qwen-2.5-32b,0.5,cot,1,0.4000,0.537584,0.4000,0.537584
3,d79cf4b97af41843834820c8c05f1e0f9f623f19,nifty-muss,bdsaglam/frames-musique-mini,default,test[:20],miprov2-light,no,bdsaglam/frames-musique-mini,default,test[20:],golden,1,qwen-2.5-32b,0.5,cot,1,0.3500,0.495338,0.3500,0.495338
4,046701ca67fe51827ed9d781ae4c45e3804e4f4f,least-lich,bdsaglam/frames-musique-mini,default,test[:20],noop,no,bdsaglam/frames-musique-mini,default,test[20:],golden,1,qwen-2.5-32b,0.0,cot,1,0.3875,0.535115,0.3875,0.535115


In [33]:
def sorted_tuple(x):
    return tuple(sorted(x))

run_counts = (
    df.groupby(param_cols[:-1])[
        "params.run"
    ]
    .aggregate(sorted_tuple)
    .reset_index()
)

In [34]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.train.dataset.path
	bdsaglam/frames-musique-mini
- params.train.dataset.name
	default
- params.train.dataset.split
	test[:20]
- params.train.optimizer
	miprov2-light
	noop
- params.train.ensemble
	no
- params.evaluation.dataset.path
	bdsaglam/frames-musique-mini
- params.evaluation.dataset.name
	default
- params.evaluation.dataset.split
	test[20:]
- params.retrieve.model_type
	golden
- params.retrieve.top_k
	1
- params.qa.model
	qwen-2.5-32b
- params.qa.temperature
	0.0
	0.5
- params.qa.technique
	cot
- params.run
	(1,)


In [35]:
df.to_json('exps.jsonl', orient='records', lines=True)

In [36]:
df[["name", "params.qa.technique", "params.train.optimizer", "metrics.exact_match", "metrics.f1"]].sort_values(by="metrics.f1", ascending=False)

,name,params.qa.technique,params.train.optimizer,metrics.exact_match,metrics.f1
1,pasty-sley,cot,miprov2-light,0.4125,0.549838
2,surer-sike,cot,noop,0.4000,0.537584
4,least-lich,cot,noop,0.3875,0.535115
3,nifty-muss,cot,miprov2-light,0.3500,0.495338
